In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.models import load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.activations import relu
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications import Xception

In [2]:
!pip install kaggle


In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d sujaykapadnis/monkeypox-skin-images-dataset-msid
!unzip -q monkeypox-skin-images-dataset-msid

 99% 54.0M/54.6M [00:02<00:00, 26.6MB/s]
100% 54.6M/54.6M [00:02<00:00, 20.5MB/s]


In [7]:
dataPath = '/content/Monkeypox Skin Images Dataset (MSID)/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'
img_height = 224
img_width = 224

dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)

Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [13]:
baseModel = Xception(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(4, activation='softmax')(dense2)


In [14]:

model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_3 (Dense)             (None, 512)               51380736  
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 72374572 (276.09 MB)
Trainable params: 515130

In [16]:

history = model.fit(
    TrainData,
    epochs=50,  # adjust the number of epochs as needed
    validation_data=validationData
)

Epoch 1/50
20/20 [==============================] - 14s 533ms/step - loss: 7.5993 - accuracy: 0.5583 - val_loss: 1.9219 - val_accuracy: 0.7566
Epoch 2/50
20/20 [==============================] - 4s 172ms/step - loss: 1.1471 - accuracy: 0.8204 - val_loss: 2.0091 - val_accuracy: 0.7171
Epoch 3/50
20/20 [==============================] - 3s 164ms/step - loss: 0.3299 - accuracy: 0.9159 - val_loss: 1.3801 - val_accuracy: 0.7500
Epoch 4/50
20/20 [==============================] - 3s 164ms/step - loss: 0.0629 - accuracy: 0.9757 - val_loss: 1.2261 - val_accuracy: 0.7500
Epoch 5/50
20/20 [==============================] - 4s 173ms/step - loss: 0.0325 - accuracy: 0.9887 - val_loss: 1.1967 - val_accuracy: 0.7895
Epoch 6/50
20/20 [==============================] - 3s 166ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.2693 - val_accuracy: 0.7763
Epoch 7/50
20/20 [==============================] - 3s 167ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.1618 - val_accuracy: 0.7961
Epoch

In [12]:
model.save("Xception.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
